# What's in this notebook?
Here we'll do some preprocessing so that we can do some NLP (tokenizing & LDA), then visualize our results.


In [133]:
import pickle
import pandas as pd
import numpy as np

In [134]:
with open('transcripts_df.pickle', 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    df = pickle.load(f)

In [135]:
df.head()

,source,transcript
0,https://www.youtube.com/watch?v=bPv21OyQLkM,I'm pleased to welcome to the platform miss Ca...
1,https://www.youtube.com/watch?v=ngzIkKtjT6o,author Tom Wolfe addressed the graduating clas...
2,https://www.youtube.com/watch?v=y5YvCbOmNxQ,ladies and gentlemen dr. Fred Rogers Wow it's ...
3,https://www.youtube.com/watch?v=Q34H3As2QJA,I'd like to tell you too true stories evening ...
4,https://www.youtube.com/watch?v=abo-YcLrnao,thank you and I'm a doctor so pay attention fo...


## Preprocessing
Removing punctuation, converting to lowercase, stop words, etc.

In [136]:
# while looking through the data, I found one of the transcripts is completely in Russian -- whoops!
# I'll see if I can explicitly specify the english youtube transcript from the youtube transcript api
from youtube_transcript_api import YouTubeTranscriptApi

df.iloc[172].transcript = ' '.join([x['text'] for x in 
                                    YouTubeTranscriptApi.get_transcript(df.iloc[172].source[-11:],
                                                                        languages=['en'])
                                   ])

In [137]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [152]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
import spacy


def preprocess(text):
    # remove punctuation
    punctuation = ['.', ',', '?', '/', "'", '"', 
                  ':', ';', ')', '(', '*', '&',
                  '^', '%', '$', '#', '!', '`', 
                   '-', '_', '[', ']', '<', '>',
                  '\n', '\r', '’', '~', '|']
    for punc in punctuation:
        text = text.replace(punc, ' ')
        
    # replace years with 'year'
    text = re.sub(r'(19|20)\d{2}\s', 'year ', text)
        
    # lowercase
    text = text.lower()
    
    # remove stopwords
    # adding 'applause' to nltk's list, since it
    # occurs a lot & is unhelpful 
    wordsFiltered = '' # collect non-stop-words
    stopWords = set(stopwords.words('english'))
    stopWords.add('applause')
    for w in word_tokenize(text):
        if w not in stopWords:
            wordsFiltered += w + ' '
    
    # lemmatize  
    # nltk's lemmatizer was garbage 
    # lemmatizer = WordNetLemmatizer() 
    # lemmatized = [lemmatizer.lemmatize(word) for word in wordsFiltered] 

    # Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
    sp = spacy.load('en')

    # Parse the sentence using the loaded 'en' model object `nlp`
    lemmatized = sp(wordsFiltered)
    
    return [word.lemma_ for word in lemmatized]

In [155]:
# takes a loooooooooooooong time
df['processed'] = df.transcript.apply(lambda x: preprocess(x) if len(x) < 100000 else 'NEED TO PREPROCESS')

In [156]:
df

,source,transcript,processed
0,https://www.youtube.com/watch?v=bPv21OyQLkM,I'm pleased to welcome to the platform miss Ca...,"[pleased, welcome, platform, miss, carlton, fi..."
1,https://www.youtube.com/watch?v=ngzIkKtjT6o,author Tom Wolfe addressed the graduating clas...,"[author, tom, wolfe, address, graduate, class,..."
2,https://www.youtube.com/watch?v=y5YvCbOmNxQ,ladies and gentlemen dr. Fred Rogers Wow it's ...,"[lady, gentlemen, dr, fred, rogers, wow, beaut..."
3,https://www.youtube.com/watch?v=Q34H3As2QJA,I'd like to tell you too true stories evening ...,"[like, tell, true, story, evening, together, m..."
4,https://www.youtube.com/watch?v=abo-YcLrnao,thank you and I'm a doctor so pay attention fo...,"[thank, doctor, pay, attention, cry, loud, tha..."
5,https://www.youtube.com/watch?v=jsHBI4w4FgQ,it is now my great pleasure to recognize today...,"[great, pleasure, recognize, today, commenceme..."
6,https://www.youtube.com/watch?v=9ZDuNzhelhQ,Will the graduates to the degree of Bachelor\n...,"[graduate, degree, bachelor, art, please, rise..."
7,https://www.youtube.com/watch?v=9LheUWrXUHU,is now my honor and privilege to present to yo...,"[honor, privilege, present, william, jefferson..."
8,https://www.youtube.com/watch?v=TvI69F8uf1A,traditionally Yale does not have a commencemen...,"[traditionally, yale, commencement, speaker, s..."
9,https://www.youtube.com/watch?v=-vsLRM4Tjd0,I asked if I got to keep the Hat they tell me ...,"[ask, get, keep, hat, tell, thank, president, ..."


Let's pickle this since it took so long!

In [160]:
with open('processed_transcripts.pickle', 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(df, f, pickle.HIGHEST_PROTOCOL)

## Vectorizing
We'll start with count vectorizing, and then we'll take a look at word2vec.

In [157]:
from sklearn.feature_extraction.text import CountVectorizer

In [159]:
vectorizer = CountVectorizer(ngram_range=(1,6))
X = vectorizer.fit_transform(df.processed.apply(lambda x: ' '.join([word for word in x])))

In [161]:
vectorizer.vocabulary_

{'pleased': 2913678,
 'welcome': 4219292,
 'platform': 2903117,
 'miss': 2466473,
 'carlton': 514502,
 'fiorina': 1326880,
 'president': 2973182,
 'chief': 578535,
 'executive': 1195084,
 'officer': 2668353,
 'hewlett': 1741202,
 'packard': 2763828,
 'company': 703599,
 'give': 1499083,
 'address': 80642,
 'good': 1567859,
 'morning': 2496949,
 'thank': 3827406,
 'much': 2519800,
 'perhaps': 2854043,
 'understatement': 4065284,
 'say': 3294633,
 'honor': 1775355,
 'back': 290165,
 'mit': 2472312,
 'graduate': 1596444,
 'class': 611711,
 'year': 4388797,
 'look': 2252425,
 'crowd': 823047,
 'humble': 1813187,
 'inspire': 1892845,
 'like': 2179419,
 'start': 3615141,
 'remark': 3177573,
 'morne': 2496933,
 'special': 3575683,
 'send': 3402527,
 'email': 1072212,
 'last': 2059823,
 'couple': 792018,
 'week': 4214083,
 'see': 3371642,
 'sit': 3483895,
 'write': 4372922,
 'speech': 3579966,
 'commencement': 684048,
 'speaker': 3571700,
 'naturally': 2562646,
 'try': 4017661,
 'figure': 1301